In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
# from imitation.data import rollout
import pickle


# import seals  # noqa: F401
import stable_baselines3 as sb3
from stable_baselines3 import PPO
from stable_baselines3.ppo import MlpPolicy

import gym
# from imitation.algorithms.bc import BC
from stable_baselines3.common.policies  import ActorCriticPolicy, ActorCriticCnnPolicy
from stable_baselines3.common.evaluation import evaluate_policy
import numpy as np

from stable_baselines3.common.env_util import make_vec_env 

/home/ns/anaconda3/lib/python3.9/site-packages/botocore/httpsession.py:34: DeprecationWarning: 'urllib3.contrib.pyopenssl' module is deprecated and will be removed in a future release of urllib3 2.x. Read more in this issue: https://github.com/urllib3/urllib3/issues/2680
  from urllib3.contrib.pyopenssl import orig_util_SSLContext as SSLContext


In [3]:
def collect_trajectory(env, policy_net, n_trajectory=20, epsilon=0.0):
    trajectories=[] 
    scores=[]
    for episode in range(n_trajectory):
        state,info = env.reset()
        score = 0 
        states=[]
        actions=[]
        rewards=[]
        while True:
            if epsilon==0:
                action, _states = policy_net.predict(state, deterministic=True)
            else:
                if np.random.random() > (1-epsilon):
                    action=env.action_space.sample() 
                else:
                    action, _states = policy_net.predict(state, deterministic=True)
            
            next_state, reward, done,s, _ = env.step(action)
            score+=reward
             
            states.append(state)
            actions.append(action)
            rewards.append(reward)
            state = next_state 
            if done or s:  
                scores.append(score)
                break
        
        states=np.vstack(states)
        actions=np.vstack(actions)
        rewards=np.vstack(rewards)
        trajectories.append((states, actions, rewards))  
        if episode % 1 == 0:
            print('{} episode score is {:.2f}'.format(episode, score))
    env.close()
    return scores, trajectories

In [4]:
savepath= "trained_models/sb3_ppo_Ant-v3_16-Sep-2023_1.zip"
model=PPO.load(savepath, print_system_info=False)

In [5]:
env_name='Ant-v3' 
env_t = gym.make(env_name)

In [6]:
print('collecting trajectories ...')
n_trajectory=10
scores, trajectories = collect_trajectory(env_t, model, n_trajectory)
mean=np.mean(scores)
print('mean score:', mean)

collecting trajectories ...
0 episode score is 4007.55
1 episode score is 4009.80
2 episode score is 3996.00
3 episode score is 4224.68
4 episode score is 4137.09
5 episode score is 3984.66
6 episode score is 3959.68
7 episode score is 4058.09
8 episode score is 1238.20
9 episode score is 4041.54
mean score: 3765.728030241264


In [11]:
import pickle

filename=f"expert_data/{env_name}_{n_trajectory}_{int(mean)}.pkl"
print('saving ',filename)

with open(filename, 'wb') as f:
    pickle.dump(trajectories, f)
print('trajectories saved.')

saving  expert_data/Ant-v3_10_3765.pkl
trajectories saved.
